# Advent of Code - 2024 - Day 20 - Problem 1

https://adventofcode.com/2024/day/20

## Load Source Data

Load the map data into `DATA`.

In [1]:
f = open("data/day20.txt", "r")
DATA = list(map(str.strip, f.readlines()))
f.close()

DATA = """###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""
DATA = list(map(str.strip, DATA.splitlines()))

DATA

['###############',
 '#...#...#.....#',
 '#.#.#.#.#.###.#',
 '#S#...#.#.#...#',
 '#######.#.#.###',
 '#######.#.#...#',
 '#######.#.###.#',
 '###..E#...#...#',
 '###.#######.###',
 '#...###...#...#',
 '#.#####.#.###.#',
 '#.#...#.#.#...#',
 '#.#.#.#.#.#.###',
 '#...#...#...###',
 '###############']

## Create Map Class

In [2]:
class Map:

    def __init__(self, lines):
        self._lines = list(map(list, DATA))
        self._rows = len(self._lines)
        self._cols = len(self._lines[0])

        # Find the starting position
        #
        for row in range(self._rows):
            for col in range(self._cols):
                if self._lines[row][col] == "S":
                    self._start = (row, col)
                if self._lines[row][col] == "E":
                    self._end = (row, col)

    def can_move(self, target, must_cheat, can_cheat):
        if must_cheat:
            return target == "#"
        elif can_cheat:
            return True
        else:
            return target != "#"

    def get_next_positions(self, position, must_cheat, can_cheat):
        row, col = position
        if row > 0:
            target = self._lines[row - 1][col]
            if self.can_move(target, must_cheat, can_cheat):
                yield (row - 1, col)
        if row < self._rows - 1:
            target = self._lines[row + 1][col]
            if self.can_move(target, must_cheat, can_cheat):
                yield (row + 1, col)
        if col > 0:
            target = self._lines[row][col - 1]
            if self.can_move(target, must_cheat, can_cheat):
                yield (row, col - 1)
        if col < self._cols - 1:
            target = self._lines[row][col + 1]
            if self.can_move(target, must_cheat, can_cheat):
                yield (row, col + 1)

    def find_paths(self, path, cheat_length, distances_to_exit):

        # print(f"find_paths - {path}")
        must_cheat = False
        can_cheat = False
        if cheat_length == len(path):
            must_cheat = True
            # print("must_cheat")
        elif cheat_length == len(path) -1:
            can_cheat = True
            # print("can_cheat")

        last_position = path[-1]
        for next_position in self.get_next_positions(last_position, must_cheat, can_cheat):
            if not next_position in path:
                new_path = list(path)
                new_path.append(next_position)
                if next_position == self._end:
                    yield new_path
                else:
                    yield from self.find_paths(new_path, cheat_length, distances_to_exit)

        # last_position = path[-1]
        # cheating = cheat_length <= len(path) <= cheat_length + 1
        # # was_cheating = cheat_length + 1 < len(path)

        # # do_pop = False
        # for next_position in self.get_next_positions(last_position, cheating):
        #     # if was_cheating:
        #     #     if next_position not in distances_to_exit: return
        #     #     if len(path) + 1 + distances_to_exit[next_position] > 10000: return

        #     if not next_position in path:
        #         new_path = list(path)
        #         new_path.append(next_position)
        #         # if do_pop: path.pop(-1)
        #         # do_pop = True
        #         # path.append(next_position)
        #         if next_position == self._end: yield new_path
        #         else: yield from self.find_paths(new_path, cheat_length, distances_to_exit)

    def print(self, path):
        m = list()
        for line in self._lines:
            m.append(list(line))

        if path != None:
            for location in path:
                row, col = location
                m[row][col] = "*"

        for line in m:
            print("".join(line))

## Create Path Class

In [3]:
class Path:

    def __init__(self, position):
        self._position = position

## Find Solutions

In [6]:
import sys

sys.setrecursionlimit(10000)

m = Map(DATA)
m.print(None)

# Find the path without cheating
#
starting_path = [m._start]
for path in m.find_paths([m._start], 1000000000, dict()):
    honest_path = path
    break

print(f"honest_path = {honest_path}")

honest_length = len(honest_path) - 1
print(f"honest_length = {honest_length}")

distances_to_exit = dict()
distance_to_exit = honest_length
for position in honest_path:
    distances_to_exit[position] = distance_to_exit
    distance_to_exit -= 1

print(f"distances_to_exit = {distances_to_exit}")

count = 0
for cheat_length in range(18, 21):  # range(1, honest_length):
    for path in m.find_paths([m._start], cheat_length, distances_to_exit):
        path_length = len(path) - 1
        # if len(path) < honest_length - 1:
        improvement = honest_length - path_length
        if improvement == 64:
            count += 1
            print("--------------------------------------------")
            print(
                f"cheat_length = {cheat_length}, path_length={path_length}, improvement = {improvement}, count = {count}"
            )
            print(path)
            m.print(path)

# print(count)

###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
honest_path = [(3, 1), (2, 1), (1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (3, 4), (3, 5), (2, 5), (1, 5), (1, 6), (1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7), (7, 8), (7, 9), (6, 9), (5, 9), (4, 9), (3, 9), (2, 9), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (2, 13), (3, 13), (3, 12), (3, 11), (4, 11), (5, 11), (5, 12), (5, 13), (6, 13), (7, 13), (7, 12), (7, 11), (8, 11), (9, 11), (9, 12), (9, 13), (10, 13), (11, 13), (11, 12), (11, 11), (12, 11), (13, 11), (13, 10), (13, 9), (12, 9), (11, 9), (10, 9), (9, 9), (9, 8), (9, 7), (10, 7), (11, 7), (12, 7), (13, 7), (13, 6), (13, 5), (12, 5), (11, 5), (11, 4), (11, 3), (12, 3), (13, 3), (13, 2), (13, 1), (12, 1), (11, 1), (10, 1), (9, 1), (9, 2), (9, 3), (8, 3), (7, 3), (7, 4), (7, 5)]
honest_len